<a href="https://colab.research.google.com/github/Ad-Chekk/AI-discord-server-bot-/blob/main/Discord_bot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%%writefile main.py
import bot



if __name__ == '__main__':
    bot.run_discord_bot()



Writing main.py


In [ ]:
%%writefile bot.py

import discord
import responses

TOKEN = 'place_your_token_here'


async def send_message(message, user_message, is_private):
    try:
        response = responses.get_response(user_message)
        await message.author.send(response) if is_private else await message.channel.send(response)

    except Exception as e:
        print(e)


def run_discord_bot():
    TOKEN = ''
    intents = discord.Intents.default()
    intents.message_content = True
    client = discord.Client(intents=intents)

    @client.event
    async def on_ready():
        print(f'{client.user} is now running!')

    @client.event
    async def on_message(message):
        if message.author == client.user:
            return

        username = str(message.author)
        user_message = str(message.content)
        channel = str(message.channel)

        print(f'{username} said: "{user_message}" ({channel})')

        if user_message[0] == '?':
            user_message = user_message[1:]
            await send_message(message, user_message, is_private=True)
        else:
            await send_message(message, user_message, is_private=False)

    client.run(TOKEN)


Overwriting bot.py


In [ ]:
%%writefile responses.py


import requests
import random
import calendar
import time
import wikipedia
from transformers import pipeline  # Import the summarizer pipeline
import language_tool_python
import nltk
from textblob import TextBlob

# Ensure necessary NLTK resources are downloaded
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')

# Create an instance for grammar checking
tool = language_tool_python.LanguageTool('en-US')

# Initialize the summarization pipeline
summarizer = pipeline("summarization", model="facebook/bart-large-cnn")

def get_response(message: str) -> str:
    define = ['wikipedia', 'wiki', 'wikihow', 'define']

    p_message = message.lower()

    if p_message == 'yolo':
        return "test.txt"

    if p_message in ['rock', 'paper', 'scissor', 'stone']:
        rcs = ['🪨Rock', '📃Paper', '✂️Scissor']
        return str(random.choice(rcs))

    if p_message == 'hello':
        return 'Hey there!😃🔥'

    if p_message == 'roll':
        return str(random.randint(1, 6))

    if p_message == 'month':
        return calendar.month(2023, 2)

    if p_message == 'password:admin':
        return '''password is correct
                    here are your confidential files:
                    example to store all files
                    https://www.google.com/
                    https://www.geeksforgeeks.com'''

    if p_message == 'link':
        return 'https://www.youtube.com/'

    if p_message == '!help':
        return ('`hey there!😃\n My commands are:\n'
                '> hello\n'
                '> roll\n'
                '> stone, paper, scissor\n'
                '> !help\n'
                '> link\n'
                '> remind "message to be displayed"\n'
                '> create file, edit file, peek file\n'
                '> password:<enter password>\n')

    msplit = p_message.split()  # Split the message into words

    if msplit[0] in define:
        search = ' '.join(msplit[2:])
        lin = int(msplit[1])
        result = wikipedia.summary(search, sentences=lin)
        return result

    if msplit[0] in ["summarize", "Summarize"]:  # Summarizer command
        try:
            text_to_summarize = ' '.join(msplit[1:])  # Get the text to summarize
            summary = summarizer(text_to_summarize, max_length=50, min_length=25, do_sample=False)
            return summary[0]['summary_text']
        except Exception as e:
            return f"Error in summarization: {e}"

    if msplit[0] == 'checkgrammar':
        text_to_check = ' '.join(msplit[1:])
        matches = tool.check(text_to_check)
        corrections = [f"Error: {match.ruleId}, Message: {match.message}, Suggestions: {match.replacements}" for match in matches]
        return "Corrections:\n" + "\n".join(corrections) if corrections else "Your grammar looks good!"

    if msplit[0] == 'sentiment':
        text_to_analyze = ' '.join(msplit[1:])
        blob = TextBlob(text_to_analyze)
        return f"Polarity: {blob.sentiment.polarity}, Subjectivity: {blob.sentiment.subjectivity}"

    if msplit[0] == 'tokenize':
        text_to_tokenize = ' '.join(msplit[1:])
        tokens = nltk.word_tokenize(text_to_tokenize)
        return f"Tokens: {tokens}"

    if msplit[0] == 'pos':
        text_to_tag = ' '.join(msplit[1:])
        tokens = nltk.word_tokenize(text_to_tag)
        tags = nltk.pos_tag(tokens)
        return f"POS Tags: {tags}"

    if msplit[0] == 'remind':  # Reminder functionality
        count = len(msplit)
        reminder_message = ' '.join(msplit[2:count])
        time.sleep(10.0)  # Sleep for 10 seconds
        return reminder_message

    return 'Sorry, I didn\'t understand what you wrote. Try typing "!help" to view all my commands.'


Writing responses.py


In [ ]:
!python main.py

2024-09-27 09:39:54.220393: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-09-27 09:39:54.255844: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-09-27 09:39:54.266578: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-09-27 09:39:54.292710: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-09-27 09:39:55.956448: W tensorflow/compiler/tf2

In [ ]:
!pip install wikipedia
!pip install transformers
!pip install language-tool-python
!pip install textblob
!pip install nltk
!pip install discord

  Preparing metadata (setup.py) ... done
  Created wheel for wikipedia: filename=wikipedia-1.4.0-py3-none-any.whl size=11679 sha256=8d3d25b3de55751fd623815e5b3a07f53f64a4f4e0e4aa23ef1a21e15a051009
  Stored in directory: /root/.cache/pip/wheels/5e/b6/c5/93f3dec388ae76edc830cb42901bb0232504dfc0df02fc50de
Successfully built wikipedia
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 22.7 MB/s eta 0:00:00


In [ ]:
!pip install discord
!pip install responses

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 11.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.7/46.7 kB 2.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.2/55.2 kB 2.7 MB/s eta 0:00:00


In [ ]:
!pip install gym==0.21.0
!pip install "gym[atari]"
!pip install tensorflow
!pip install ale-py==0.7.4
!pip install autorom[accept-rom-license]

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 10.9 MB/s eta 0:00:00
  error: subprocess-exited-with-error
  
  × python setup.py egg_info did not run successfully.
  │ exit code: 1
  ╰─> See above for output.
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
  Preparing metadata (setup.py) ... error
error: metadata-generation-failed

× Encountered error while generating package metadata.
╰─> See above for output.

note: This is an issue with the package mentioned above, not pip.
hint: See above for details.
INFO: pip is looking at multiple versions of gym[atari] to determine which version is compatible with other requirements. This could take a while.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 721.7/721.7 kB 17.8 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 35.7 MB/s eta 0: